<a href="https://colab.research.google.com/github/JeonghanHong/eCRF/blob/main/eCRF_Fitbit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install streamlit pyngrok pandas matplotlib statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok

# ngrok 인증 토큰 설정  2blRD9q8ERUslzqflw7XLOMdgqb_67NNKcTXEz9BYyBEBvUpy
ngrok.set_auth_token("2blRD9q8ERUslzqflw7XLOMdgqb_67NNKcTXEz9BYyBEBvUpy")

# 이후 ngrok를 사용하여 터널 생성

In [ ]:
%%writefile app.py

# 필요한 라이브러리 가져오기
import streamlit as st
import os
from glob import glob
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import AutoDateLocator
from statsmodels.nonparametric.smoothers_lowess import lowess

# Directory path
dir_path = '/content/drive/MyDrive/Fitbit/eCRF'

# Retrieve all CSV filenames in the directory
csv_files = glob(f'{dir_path}/S*.csv')

all_subject_ids = [os.path.splitext(os.path.basename(f))[0] for f in csv_files]

# Load the Demo.csv file
demo_df = pd.read_csv('/content/drive/MyDrive/Fitbit/eCRF/Demo.csv')

# Sidebar for user input
with st.sidebar:
    # Class selection
    class_options = demo_df['Class'].unique()
    selected_class = st.selectbox('Select Class', class_options)

    # Filter demo_df based on selected Class
    class_filtered_df = demo_df[demo_df['Class'] == selected_class]

    # SubClass selection
    subclass_options = class_filtered_df['SubClass'].unique()
    selected_subclass = st.selectbox('Select SubClass', subclass_options)

    # Filter class_filtered_df based on selected SubClass
    subclass_filtered_df = class_filtered_df[class_filtered_df['SubClass'] == selected_subclass]

    # Subject ID selection
    selected_subject_ids = st.multiselect('Select Subject ID', subclass_filtered_df['Subject_ID'].tolist())

    # Subject_ID 선택에 따른 Start Date와 End Date 디폴트 값 설정
    if selected_subject_ids:
        selected_demo_records = subclass_filtered_df[subclass_filtered_df['Subject_ID'].isin(selected_subject_ids)]

        if not selected_demo_records.empty:
            study_date = pd.to_datetime(selected_demo_records['Study_Date'].min())
            start_date_default = study_date + pd.DateOffset(days=14)
            end_date_default = study_date + pd.DateOffset(days=20)
        else:
            start_date_default = datetime.today()
            end_date_default = datetime.today()
    else:
        start_date_default = datetime.today()
        end_date_default = datetime.today()

    # User input for the start date and end date
    start_date = st.date_input("Select Start Date", start_date_default)
    end_date = st.date_input("Select End Date", end_date_default)


    # 나머지 코드...

    # 날짜 입력 검증
    if start_date > end_date:
        st.error('End Date must be later than Start Date.')
        st.stop()

    # Slider for adjusting LOESS smoothing fraction
    st.write('Adjust "frac" for LOESS smoothing (0.001 to 0.1)')
    frac_value = st.slider("", 0.001, 0.1, 0.05)

# Add the following code to display the selected Subject ID's information
if selected_subject_ids:
    selected_demo_records = demo_df[demo_df['Subject_ID'].isin(selected_subject_ids)]

    st.write(f"Selected Subject IDs: {', '.join(selected_subject_ids)}")
    st.write(selected_demo_records)

# Instructional message
if not selected_subject_ids:
    st.markdown("""
    **This application displays LOESS smoothed data for calories and HRV_Score and calculates the cross-correlation between these two time series.**

    Please select 'Subject ID' from the left tab, and adjust the 'Start Date' and 'End Date' for the analysis, and "frac" for LOESS smoothing accordingly.

    **Sidebar:**
    - A date input widget labeled "Select Start Date" for the user to choose the starting date for the data analysis.
    - A date input widget labeled "Select End Date" for the user to choose the ending date for the data analysis.
    - A slider labeled "Adjust 'frac' for LOESS smoothing (0.01 to 0.1)" allowing the user to adjust the fraction for LOESS smoothing, with a range from 0.001 to 0.1 and a default value set at 0.05.
    - A multi-select dropdown list labeled "Select Subject ID" populated with all the unique subject IDs. The user can select one or more subject IDs from this list for data analysis.

    **Main Panel:**
    - Initially, the main panel may not display any data until the user selects at least one Subject ID from the sidebar.
    - Once the Subject IDs are selected, the main panel will display the DataFrame containing data corresponding to the selected Subject IDs and date range.
    - After the data is loaded and filtered based on the user's selections, any further analysis or plots (like the LOESS smoothed time series or cross-correlation results) will be shown here as well.
    """, unsafe_allow_html=True)
    st.stop()

# Load and concatenate selected data files
selected_data_df = pd.DataFrame()
for subject_id in selected_subject_ids:
    file_path = f"{dir_path}/{subject_id}.csv"
    temp_df = pd.read_csv(file_path, parse_dates=['DateTime'])

    temp_df.rename(columns={
        'Heart_Rat': 'Heart_Rate',
        'SDNN': 'SDNN',
        'RMSSD': 'RMSSD',
        'Subject_ID_x': 'Subject_ID',
        'VLF': 'VLF',
        'LF': 'LF',
        'HF': 'HF',
        'LFHF_Ratio': 'LFHF_Ratio',
        'Subject_ID_y': 'Subject_ID_y',
        'Level_x': 'Activity',
        'Mets': 'Mets',
        'Value': 'Calories',
        'Level_y': 'Sleep',
    }, inplace=True)

    selected_data_df = pd.concat([selected_data_df, temp_df], ignore_index=True)







# Define a mapping dictionary for Sleep quality
sleep_mapping = {
    '': -1,  # Add mapping for blank entries
    'None': -1,
    'restless': 0,
    'wake': 1,
    'asleep': 2,
    'awake': 3,
    'light': 4,
    'rem': 5,
    'deep': 6
}

# Map the Sleep column to numeric values based on the mapping
selected_data_df['Sleep'] = selected_data_df['Sleep'].map(sleep_mapping).fillna(-1).astype(int)


# Adjust the start_date and end_date to cover the entire day
start_datetime = datetime.combine(start_date, datetime.min.time())
end_datetime = datetime.combine(end_date, datetime.max.time())

# Filter the data based on the selected date range including the time
selected_data_df = selected_data_df[
    (selected_data_df['DateTime'] >= start_datetime) &
    (selected_data_df['DateTime'] <= end_datetime)
]

# Set 'DateTime' as index
selected_data_df.set_index('DateTime', inplace=True)


# 선택된 모든 subject_id에 대한 데이터 프레임을 기본적으로 표시
for subject_id in selected_subject_ids:
    st.write(f"Data for Subject ID: {subject_id}")
    subject_data = selected_data_df[selected_data_df['Subject_ID'] == subject_id]
    columns_to_display = [col for col in subject_data.columns if col not in ['Subject_ID', 'Subject_ID_y', 'Seconds']]
    st.dataframe(subject_data[columns_to_display])



# Optionally display the DataFrame without 'Subject_ID', 'Subject_ID_y', and 'Seconds' columns
#if st.checkbox("Show DataFrame"):
#    for subject_id in selected_subject_ids:
#        st.write(f"Data for Subject ID: {subject_id}")
#        subject_data = selected_data_df[selected_data_df['Subject_ID'] == subject_id]
#        columns_to_display = [col for col in subject_data.columns if col not in ['Subject_ID', 'Subject_ID_y', 'Seconds']]
#        st.dataframe(subject_data[columns_to_display + ['Sleep']])  # Include the 'Sleep' column
#else:
#    # If "Show DataFrame" is not checked, display the combined DataFrame
#    columns_to_display = [col for col in selected_data_df.columns if col not in ['Subject_ID', 'Subject_ID_y', 'Seconds']]
#    st.dataframe(selected_data_df[columns_to_display])

# Ensure the selected DataFrame has the necessary columns and is not empty
if not selected_data_df.empty and {'RMSSD', 'Calories', 'Sleep'}.issubset(selected_data_df.columns):
    # Apply LOESS smoothing to the selected data
    hrv_score_lowess = lowess(selected_data_df['RMSSD'], mdates.date2num(selected_data_df.index), frac=frac_value)
    calories_lowess = lowess(selected_data_df['Calories'], mdates.date2num(selected_data_df.index), frac=frac_value)

    # Convert the lowess outputs to DataFrames
    hrv_score_lowess_df = pd.DataFrame(hrv_score_lowess, columns=['DateTimeNum', 'HRV_Score_LOESS'])
    hrv_score_lowess_df['DateTime'] = mdates.num2date(hrv_score_lowess_df['DateTimeNum'])
    hrv_score_lowess_df.set_index('DateTime', inplace=True)

    calories_lowess_df = pd.DataFrame(calories_lowess, columns=['DateTimeNum', 'Calories_LOESS'])
    calories_lowess_df['DateTime'] = mdates.num2date(calories_lowess_df['DateTimeNum'])
    calories_lowess_df.set_index('DateTime', inplace=True)


    # UTC 시간대로 인덱스 변환 후 다시 로컬 시간대로 인덱스 변환
    selected_data_df.index = selected_data_df.index.tz_localize('UTC')
    selected_data_df.index = selected_data_df.index.tz_convert('Asia/Seoul')
    # Combine the HRV Score and Calories DataFrames on their index
    combined_df = hrv_score_lowess_df.join(calories_lowess_df['Calories_LOESS'])

    # 'Sleep' 데이터를 'combined_df'에 추가
    combined_df['Sleep'] = selected_data_df['Sleep']




    # Create two subplots vertically
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 14))

    # First subplot for HRV Score vs. Calories
    ax1.scatter(
        combined_df.index,
        combined_df['HRV_Score_LOESS'],
        label='HRV Score (LOESS)',
        color='darkblue',
        alpha=0.5,
        s=5
    )
    ax1.set_ylabel('HRV Score', color='darkblue')
    ax1.tick_params(axis='y', labelcolor='darkblue')
    ax1.set_title('HRV Score vs. Calories', fontsize=16)
    ax1.xaxis.set_major_locator(AutoDateLocator(minticks=10, maxticks=20))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    ax1.set_xlim(left=start_datetime)
    ax1.legend(loc='upper left')
    # 숨기기를 원하는 스파인 설정
    ax1.spines['top'].set_visible(False)



    # Create a secondary y-axis for the Calories data
    ax1b = ax1.twinx()
    ax1b.scatter(
        combined_df.index,
        combined_df['Calories_LOESS'],
        label='Calories (LOESS)',
        color='#654321',
        alpha=0.5,
        s=5
    )
    ax1b.set_ylabel('Calories', color='#654321')
    ax1b.tick_params(axis='y', labelcolor='#654321')
    ax1b.legend(loc='upper right')
    # 숨기기를 원하는 스파인 설정
    ax1b.spines['top'].set_visible(False)
    # Second subplot for HRV Score vs. Sleep (with similar settings)
    ax2.scatter(
        combined_df.index,
        combined_df['HRV_Score_LOESS'],
        label='HRV Score (LOESS)',
        color='darkblue',
        alpha=0.5,
        s=5
    )
    ax2.set_ylabel('HRV Score', color='darkblue')
    ax2.tick_params(axis='y', labelcolor='darkblue')
    ax2.set_title('HRV Score vs. Sleep', fontsize=16)
    ax2.xaxis.set_major_locator(AutoDateLocator(minticks=10, maxticks=20))
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    ax2.set_xlim(left=start_datetime)
    ax2.legend(loc='upper left')
    # 숨기기를 원하는 스파인 설정
    ax2.spines['top'].set_visible(False)




    # Create a secondary y-axis for the Sleep data
    ax2b = ax2.twinx()
    ax2b.scatter(
        combined_df.index,
        combined_df['Sleep'],
        label='Sleep',
        color='red',  # Sleep 색상 (red로 설정)
        alpha=0.7,
        s=3
    )


    ax2b.set_ylabel('Sleep', color='red')
    ax2b.tick_params(axis='y', labelcolor='red')
    ax2b.legend(loc='upper right')
    # 숨기기를 원하는 스파인 설정
    ax2b.spines['top'].set_visible(False)
    ax2b.spines['bottom'].set_visible(False)
    # Set y-axis limits for Sleep data
    ax2b.set_ylim(-1, 6)  # Set the y-axis limits from -1 to 6

    # Set y-axis tick positions and labels for Sleep data
    sleep_labels = ['None', 'restless', 'wake', 'asleep', 'awake', 'light', 'rem', 'deep']
    ax2b.set_yticks(range(-1, 7))  # Set the y-axis tick positions
    ax2b.set_yticklabels(sleep_labels)  # Set the y-axis tick labels to sleep_labels

    # Remove grid from both subplots
    ax1.grid(False)
    ax2.grid(False)
    ax2.spines['top'].set_visible(False)

    # Format the plot
    fig.autofmt_xdate(rotation=90)
    plt.tight_layout()  # Adjust the layout
    # Display the plot in Streamlit
    st.pyplot(fig)

    # Define a function to calculate cross-correlation
    def cross_correlation(timeseries1, timeseries2, lag=24, plot=True):
        ccf = [timeseries1.corr(timeseries2.shift(t)) for t in range(-lag, lag+1)]
        if plot:
            fig, ax = plt.subplots(figsize=(12, 5))
            ax.plot(range(-lag, lag+1), ccf)
            ax.axhline(0, linestyle='--', color='k')
            ax.axvline(0, linestyle='--', color='k')
            ax.set_title('Investigating Lead-Lag Dynamics Between Calories and HRV Score Over 6 Hours')
            ax.set_xlabel('Lag in 5-minute intervals')
            ax.set_ylabel('Correlation coefficient')
            st.pyplot(fig)
        return ccf

    # Define the interpret_correlation function
    def interpret_correlation(max_corr, lag, leading_var, lagged_var, interval_minutes=5):
        time_diff = abs(lag) * interval_minutes

        if lag > 0:
            relationship = f"{lagged_var} increases, followed by an increase in {leading_var} after approximately {time_diff} minutes."
        elif lag < 0:
            relationship = f"{leading_var} increases, followed by an increase in {lagged_var} after approximately {time_diff} minutes."
        else:
            relationship = "both variables change simultaneously."

        interpretation = (
            f"The maximum cross-correlation value is {max_corr:.4f}, occurring at a lag of {lag} intervals. "
            f"This suggests that {relationship}"
        )
        return interpretation

    # Calculate cross-correlation and capture the result
    cross_correlation_result = cross_correlation(combined_df['Calories_LOESS'], combined_df['HRV_Score_LOESS'], lag=24)

    # Find the index of the maximum correlation coefficient in the result
    max_corr_index = max(range(len(cross_correlation_result)), key=lambda index: abs(cross_correlation_result[index]))
    max_corr_value = cross_correlation_result[max_corr_index]
    max_lag = max_corr_index - 24

    if max_lag > 0:
        leading = "HRV_Score"
        lagged = "Calories"
    elif max_lag < 0:
        leading = "Calories"
        lagged = "HRV_Score"
    else:
        leading = lagged = "none; the variables change simultaneously"

    interpretation = interpret_correlation(max_corr_value, max_lag, leading, lagged)
    st.write(f"Maximum cross-correlation value: {max_corr_value:.4f} at lag: {max_lag}")
    st.write(f"Leading variable: {leading}")
    st.write(f"Lagged variable: {lagged}")
    st.write(interpretation)



#........Insights...................................................................................

import pandas as pd
import streamlit as st
import pytz  # Import the pytz library for timezone conversion

# Load your data into combined_df and set start_datetime and end_datetime

# Calculate the total time spent in deep sleep (assuming your Sleep column has values 6 for deep sleep)
total_deep_sleep_time = combined_df['Sleep'][combined_df['Sleep'] == 6].count() * 5  # Assuming 5 minutes per interval

# Calculate the total time within the selected time range
total_selected_time = (end_datetime - start_datetime).total_seconds() / 3600  # Convert to hours

# Calculate the maximum and minimum values for HRV (RMSSD) and Calories
max_hrv = combined_df['HRV_Score_LOESS'].max()
min_hrv = combined_df['HRV_Score_LOESS'].min()
max_calories = combined_df['Calories_LOESS'].max()
min_calories = combined_df['Calories_LOESS'].min()

# Calculate the deep sleep percentage for each hour interval
hourly_deep_sleep_percentage = []
for hour in range(24):
    hour_start = start_datetime.replace(hour=hour, minute=0, second=0)
    hour_end = start_datetime.replace(hour=hour, minute=59, second=59)

    # Convert hour_start and hour_end to 'Asia/Seoul' timezone
    seoul_timezone = pytz.timezone('Asia/Seoul')
    hour_start_seoul = hour_start.astimezone(seoul_timezone)
    hour_end_seoul = hour_end.astimezone(seoul_timezone)

    hour_data = combined_df[(combined_df.index >= hour_start_seoul) & (combined_df.index <= hour_end_seoul)]

    # Calculate deep sleep percentage for this hour
    deep_sleep_time = hour_data['Sleep'][hour_data['Sleep'] == 6].count() * 5
    hour_percentage = (deep_sleep_time / 60) * 100  # Assuming 60 minutes in an hour
    hourly_deep_sleep_percentage.append(hour_percentage)

# Find the hour with the highest deep sleep percentage
max_deep_sleep_hour = hourly_deep_sleep_percentage.index(max(hourly_deep_sleep_percentage))
max_deep_sleep_percentage = max(hourly_deep_sleep_percentage)

# Find the time when maximum and minimum HRV (RMSSD) values occurred
max_hrv_time = combined_df[combined_df['HRV_Score_LOESS'] == max_hrv].index[0]
min_hrv_time = combined_df[combined_df['HRV_Score_LOESS'] == min_hrv].index[0]

# Find the time when maximum and minimum Calories values occurred
max_calories_time = combined_df[combined_df['Calories_LOESS'] == max_calories].index[0]
min_calories_time = combined_df[combined_df['Calories_LOESS'] == min_calories].index[0]

# Convert timestamps to 'Asia/Seoul' timezone
seoul_timezone = pytz.timezone('Asia/Seoul')
max_hrv_time_seoul = max_hrv_time.astimezone(seoul_timezone)
min_hrv_time_seoul = min_hrv_time.astimezone(seoul_timezone)
max_calories_time_seoul = max_calories_time.astimezone(seoul_timezone)
min_calories_time_seoul = min_calories_time.astimezone(seoul_timezone)

# Calculate the corresponding hour start and end times
max_deep_sleep_hour_start = start_datetime.replace(hour=max_deep_sleep_hour, minute=0, second=0)
max_deep_sleep_hour_end = start_datetime.replace(hour=max_deep_sleep_hour, minute=59, second=59)

# Convert hour start and end times to 'Asia/Seoul' timezone
max_deep_sleep_hour_start_seoul = max_deep_sleep_hour_start.astimezone(seoul_timezone)
max_deep_sleep_hour_end_seoul = max_deep_sleep_hour_end.astimezone(seoul_timezone)


# Create a DataFrame to organize the results
results_data = {
    "Metric": ["HRV Max at", "HRV Min at", "Calories Max at", "Calories Min at", "Most Deep Sleep % at"],
    "When": [max_hrv_time_seoul.strftime('%Y-%m-%d %H:%M'), min_hrv_time_seoul.strftime('%Y-%m-%d %H:%M'), max_calories_time_seoul.strftime('%Y-%m-%d %H:%M'), min_calories_time_seoul.strftime('%Y-%m-%d %H:%M'), f"{max_deep_sleep_hour_start_seoul.strftime('%Y-%m-%d %H:%M')} - {max_deep_sleep_hour_end_seoul.strftime('%Y-%m-%d %H:%M')}"],
    "Value": [f"{max_hrv:.4f}", f"{min_hrv:.4f}", f"{max_calories:.4f}", f"{min_calories:.4f}", f"{max_deep_sleep_percentage:.4f}%"]
}

# Create a DataFrame
results_df = pd.DataFrame(results_data)

# Display the results in a table with download, search, and expand capabilities
st.write("##### Insights:")
st.dataframe(results_df)


# ..........HRV Metrics....................................................

# 'DateTime' 열을 이미 앞에서 인덱스로 설정


# Define the classification logic
def determine_patient_status(row):
    # 스트레스 상태 판별
    if row['RMSSD'] < 30:
        stress_status = "Low Stress"  # 저 스트레스
    else:
        stress_status = "High Stress"  # 고 스트레스

    # 신경계 판별 (Nervous System Status)
    if row['LFHF_Ratio'] > 1.5:
        nervous_system_status = "Sympathetic Dominance"  # 교감신경우세
    elif row['LFHF_Ratio'] < 0.5:
        nervous_system_status = "Parasympathetic Dominance"  # 부교감신경우세
    else:
        nervous_system_status = "Balanced Autonomic"  # 균형잡힌 자율신경계

    return stress_status, nervous_system_status

# 'DateTime' 열을 인덱스로 유지하면서 상태 분류 적용
selected_data_df[['Stress Status', 'Nervous System Status']] = selected_data_df.apply(
    lambda row: pd.Series(determine_patient_status(row)), axis=1
)

final_df = selected_data_df


# Streamlit app code
columns_to_display = ['Stress Status', 'Nervous System Status', 'Heart_Rate', 'SDNN', 'RMSSD', 'VLF', 'LF', 'HF', 'LFHF_Ratio']

# 소수점 네 번째 자리까지 표시하도록 옵션 설정
pd.options.display.float_format = '{:.4f}'.format
st.write("##### HRV Metrics:")
st.dataframe(final_df[columns_to_display])











#------Stress Analytics-------------------------------------------------------------------------

# Define the URL for the image
image_url = "https://raw.githubusercontent.com/JeonghanHong/eCRF/main/stress.png"

def get_transition_message(stress_status_before, nervous_system_status_before, stress_status_after, nervous_system_status_after):
    # Define messages for each case, with corrections
    messages = {
        ('Low Stress', 'Parasympathetic Dominance', 'High Stress', 'Sympathetic Dominance'):
        """Change from [Low Stress and Parasympathetic Dominance] to [High Stress and Sympathetic Dominance]

Cause: Tension, stress, or sudden psychological shock.

Diagnosis: Stress levels increase with activation of the sympathetic nervous system, leading to a heightened state of tension in the body.

Prescription: The use of Theanine and Tryptophan can aid in stress management and tension relief. Additionally, supplementing with Pyridoxine (Vitamin B6) can enhance stress management and improve nerve function.

These measures serve as general guidelines for managing stress and maintaining physical well-being. However, it is important to consult with a healthcare professional for personalized advice based on individual circumstances and needs.""",

        ('Low Stress', 'Sympathetic Dominance', 'High Stress', 'Sympathetic Dominance'):
        """Change from [Low Stress and Sympathetic Dominance] to [High Stress and Sympathetic Dominance]

Cause: Sudden stressors, tension, or urgent situations.

Diagnosis: Stress levels increase with activation of the sympathetic nervous system, resulting in heightened alertness and tension.

Prescription: Theanine can be used to alleviate stress and tension and prepare for acute stress situations. Additionally, supplementing with Magnesium and Potassium can support the sympathetic nervous system and promote overall bodily stability.

These measures serve as general guidelines for managing stress and maintaining physical well-being. However, it is important to consult with a healthcare professional for personalized advice based on individual circumstances and needs.""",

        # Add the image URL here for the specific case
        ('Low Stress', 'Sympathetic Dominance', 'High Stress', 'Parasympathetic Dominance'):
        f"""Change from [Low Stress and Sympathetic Dominance] to [High Stress and Parasympathetic Dominance]

Cause: Sudden stressful situations, tension, or urgent scenarios.

Diagnosis: Stress levels increase with activation of the sympathetic nervous system, transitioning to an activated parasympathetic nervous system, inducing a state of relaxation.

Prescription: The use of Theanine, Tryptophan, and Tyrosine can stabilize emotions during stress and activate the parasympathetic nervous system. Additionally, supplementing with Iodine, Zinc, and Manganese can maintain mineral balance and support metabolism.

These measures serve as general guidelines for managing stress and maintaining physical well-being. However, it is important to consult with a healthcare professional for personalized advice based on individual circumstances and needs.""",

        ('Low Stress', 'Balanced Autonomic', 'High Stress', 'Parasympathetic Dominance'):
        """Change from [Low Stress and Balanced Autonomic] to [High Stress and Parasympathetic Dominance]

Cause: Tension, stress, or sudden psychological shock.

Diagnosis: Stress levels increase dramatically with activation of the parasympathetic nervous system, inhibiting the transition to a state of rest and instead leading to increased secretion of pancreatic cells.

Prescription: To aid in stress management and tension relief, Theanine and Tryptophan can be utilized. Additionally, supplementing with Iodine, Zinc, and Manganese can support mineral balance and metabolism.

These measures serve as general guidelines for managing stress and maintaining physical well-being. However, it is important to consult with a healthcare professional for personalized advice based on individual circumstances and needs.""",

        ('Low Stress', 'Balanced Autonomic', 'High Stress', 'Sympathetic Dominance'):
        """Change from [Low Stress and Balanced Autonomic] to [High Stress and Sympathetic Dominance]

Cause: Sudden stressors, tension, or urgent situations.

Diagnosis: Stress levels increase significantly with activation of the sympathetic nervous system, leading to a state of heightened physical activity instead of relaxation.

Prescription: To prepare for acute stress situations, Theanine can be used to alleviate stress and tension. Supplementing with Magnesium and Potassium can support the sympathetic nervous system and promote overall bodily stability.

These measures serve as general guidelines for managing stress and maintaining physical well-being. However, it is important to consult with a healthcare professional for personalized advice based on individual circumstances and needs.""",
    }

    # Lookup the message based on the transition
    key = (stress_status_before, nervous_system_status_before, stress_status_after, nervous_system_status_after)
    message = messages.get(key, "No specific case matched.")

    # Add the image to the message if it's one of the specified cases
    if key in messages:
        st.image(image_url, caption="Stress Image", use_column_width=True)

    return message

# Call the function to display the message with the image for the specified case
#message = get_transition_message('Low Stress', 'Sympathetic Dominance', 'High Stress', 'Parasympathetic Dominance')

# Display the message
#st.write(message)





def detect_and_display_transitions(df):
    previous_status = ('', '')
    transition_start_time = None
    longest_transition = None
    longest_duration = timedelta(0)

    for index, row in df.iterrows():
        current_status = (row['Stress Status'], row['Nervous System Status'])

        # Start tracking a new transition
        if current_status == ('High Stress', 'Sympathetic Dominance') and previous_status == ('Low Stress', 'Sympathetic Dominance'):
            transition_start_time = index  # Update the start time of the transition

        # Check if the current status is different from the target transition status
        elif current_status != ('High Stress', 'Sympathetic Dominance') and transition_start_time:
            duration = index - transition_start_time
            if duration > longest_duration:
                longest_duration = duration
                longest_transition = (transition_start_time, index, longest_duration)
            transition_start_time = None  # Reset start time for next transition

        previous_status = current_status

    if longest_transition:
        start, end, duration = longest_transition
        st.write(f"Significant state transition detected from {start} to {end}, lasting {duration}.")
        # Assuming you have a predefined function to get a transition message
        transition_message = get_transition_message('Low Stress', 'Sympathetic Dominance', 'High Stress', 'Sympathetic Dominance')
        st.markdown(transition_message)
    else:
        st.write("No significant state transition was detected.")

# Assuming selected_data_df is a DataFrame structured with the necessary columns
detect_and_display_transitions(selected_data_df)




#.....Sleep_metrics................................................
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta, time
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta, time

# Define the boundary for a new day and inactive hours
day_start_hour = 9
inactive_start, inactive_end = time(9, 0), time(21, 0)



# Function to assign session ids based on transitions, considering inactive periods
def assign_sleep_sessions(sleep_data, index):
    sleep_sessions = []
    session_id = 0
    for i, (timestamp, sleep_state) in enumerate(zip(index, sleep_data)):
        # Increment session_id at the start of a new sleep period
        if i > 0 and sleep_state != -1 and sleep_data[i - 1] == -1:
            session_id += 1
        # Consider inactive hours as separate 'sessions'
        if inactive_start <= timestamp.time() <= inactive_end:
            session_id += 1
        sleep_sessions.append(session_id)
    return sleep_sessions

# Adjust day based on a 09:00 threshold
def get_sleep_day(dt):
    if dt.time() < inactive_start:
        return (dt - timedelta(days=1)).date()
    return dt.date()

# Main function to calculate sleep metrics
def calculate_sleep_metrics(df):
    df = df.copy()
    if 'Seconds' in df.columns:
        df = df.drop(columns=['Seconds'])
    df.index = pd.to_datetime(df.index).tz_localize(None)
    df['Sleep_Session'] = assign_sleep_sessions(df['Sleep'], df.index)
    df['Sleep_Day'] = df.index.map(get_sleep_day)
    daily_sleep_metrics = []

    for sleep_day, day_group in df.groupby('Sleep_Day'):
        filtered_group = day_group[(day_group.index.time < inactive_start) | (day_group.index.time > inactive_end)]
        for session, session_group in filtered_group.groupby('Sleep_Session'):
            if session_group.empty:
                continue
            nominal_sleep_duration_minutes = session_group.index.to_series().diff().dt.total_seconds().dropna().sum() / 60
            net_sleep_duration_minutes = session_group[~session_group['Sleep'].isin([0, 1, 2, 3])].index.to_series().diff().dt.total_seconds().dropna().sum() / 60
            sleep_efficiency = (net_sleep_duration_minutes / nominal_sleep_duration_minutes) * 100 if nominal_sleep_duration_minutes > 0 else 0
            deep_sleep = session_group[session_group['Sleep'] == 6].index.to_series().diff().dt.total_seconds().sum() / 60
            rem_sleep = session_group[session_group['Sleep'] == 5].index.to_series().diff().dt.total_seconds().sum() / 60
            light_sleep = session_group[session_group['Sleep'] == 4].index.to_series().diff().dt.total_seconds().sum() / 60
            total_measured_sleep = deep_sleep + rem_sleep + light_sleep
            deep_percentage = (deep_sleep / total_measured_sleep * 100) if total_measured_sleep > 0 else 0
            rem_percentage = (rem_sleep / total_measured_sleep * 100) if total_measured_sleep > 0 else 0
            light_percentage = (light_sleep / total_measured_sleep * 100) if total_measured_sleep > 0 else 0
            daily_sleep_metrics.append({
                'Date': sleep_day,
                'Nominal Sleep (min)': nominal_sleep_duration_minutes,
                'Net Sleep (min)': net_sleep_duration_minutes,
                'Sleep Efficiency (%)': f"{sleep_efficiency:.2f}",
                'Sleep Composition (Deep:REM:Light)': f"{deep_percentage:.0f}%:{rem_percentage:.0f}%:{light_percentage:.0f}%"
            })
    return pd.DataFrame(daily_sleep_metrics)



import pandas as pd
import streamlit as st

# Analyze sleep data function
def analyze_sleep_data(df):
    # Calculate averages and distributions
    average_nominal_sleep = df['Nominal Sleep (min)'].mean()
    average_net_sleep = df['Net Sleep (min)'].mean()
    avg_nominal_hrs, avg_nominal_mins = divmod(average_nominal_sleep, 60)
    avg_net_hrs, avg_net_mins = divmod(average_net_sleep, 60)
    min_sleep = df['Net Sleep (min)'].min()
    max_sleep = df['Net Sleep (min)'].max()

    # Convert sleep efficiency percentages to numeric, skipping NA
    df['Sleep Efficiency (%)'] = pd.to_numeric(df['Sleep Efficiency (%)'].str.replace('%', ''), errors='coerce')
    average_efficiency = df['Sleep Efficiency (%)'].mean(skipna=True)

    # Process sleep composition
    sleep_composition = df['Sleep Composition (Deep:REM:Light)'].str.replace('%', '').str.split(':', expand=True).apply(pd.to_numeric, errors='coerce')
    avg_deep, avg_rem, avg_light = sleep_composition.mean()

    # Prepare interpretation string
    interpretation = """
    [Sleep Duration] During the observed period, the average nominal sleep duration was {0} hours and {1} minutes, with a sleep time distribution ranging from {2} to {3} minutes.<br><br>
    [Sleep Efficiency] The average sleep efficiency was {4:.2f}%, which is generally considered good.<br><br>
    [Sleep Composition] However, the average sleep composition ratios of Deep:REM:Light were {5:.0f}%:{6:.0f}%:{7:.0f}%, indicating a lower than ideal proportion of deep to REM sleep according to standard sleep criteria.
    """.format(int(avg_nominal_hrs), int(avg_nominal_mins), min_sleep, max_sleep, average_efficiency, avg_deep, avg_rem, avg_light)

    # Display the interpretation in Streamlit
    st.markdown(interpretation, unsafe_allow_html=True)



# Calculate sleep metrics
daily_sleep_metrics_df = calculate_sleep_metrics(combined_df)

# Assuming daily_sleep_metrics_df is the DataFrame containing all the metrics you've calculated
filtered_daily_sleep_metrics_df = daily_sleep_metrics_df[
    ~((daily_sleep_metrics_df['Sleep Composition (Deep:REM:Light)'] == '0%:0%:0%') )
]

filtered_daily_sleep_metrics_df = filtered_daily_sleep_metrics_df.reset_index(drop=True)

# Now display the DataFrame without the index
st.write("##### Sleep Metrics:")
st.dataframe(filtered_daily_sleep_metrics_df)

# Call the function to analyze the sleep data
analyze_sleep_data(filtered_daily_sleep_metrics_df)




#------- Sleep Analytics Graph-----------------------------------------------------------------------------------------
import matplotlib.pyplot as plt

# Ensure 'Date' is a datetime object and set as index
if not isinstance(daily_sleep_metrics_df.index, pd.DatetimeIndex):
    daily_sleep_metrics_df['Date'] = pd.to_datetime(daily_sleep_metrics_df['Date'])
    daily_sleep_metrics_df.set_index('Date', inplace=True)

# Filter out dates where 'Nominal Sleep (min)' is 200 minutes or less
filtered_df = daily_sleep_metrics_df[daily_sleep_metrics_df['Nominal Sleep (min)'] > 200]

# Create a new figure and axis object
fig, ax = plt.subplots(figsize=(10, 6))

# Plot Nominal Sleep
ax.bar(filtered_df.index, filtered_df['Nominal Sleep (min)'], label='Nominal Sleep (min)', color='skyblue')

# Plot Net Sleep within Nominal Sleep
ax.bar(filtered_df.index, filtered_df['Net Sleep (min)'], label='Net Sleep (min)', color='blue')

# Customize the x-axis to show both date and day of the week
ax.set_xticks(filtered_df.index)
ax.set_xticklabels(filtered_df.index.strftime('%Y-%m-%d (%a)'), rotation=90, ha='right')

# Set the labels and title
ax.set_xlabel('Date and Day of the Week')
ax.set_ylabel('Sleep (min)')
ax.set_title('Sleep Analysis')

# Add a legend
ax.legend()

# Show the plot
plt.tight_layout()
st.pyplot(fig)






Writing app.py


In [ ]:
!nohup streamlit run app.py &
from pyngrok import ngrok

# 기존 터널이 있다면 종료
ngrok.kill()

# Streamlit이 사용하는 포트 (기본값: 8501)에 대한 퍼블릭 URL 생성
public_url = ngrok.connect("8501")
print("Streamlit 애플리케이션의 퍼블릭 URL:", public_url)


nohup: appending output to 'nohup.out'
Streamlit 애플리케이션의 퍼블릭 URL: NgrokTunnel: "https://5e07-34-86-152-4.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!ngrok diagnose

Testing ngrok connectivity...

Internet Connectivity
  Name Resolution                           [ OK ]
  TCP                                       [ OK ]
  TLS                                       [ OK ]
Localhost Connectivity
  Name Resolution                           [ OK ]
Ngrok Connectivity - Region: United States
  Name Resolution                           [ OK ]
  TCP                                       [ OK ]
  TLS                                       [ OK ]
  Tunnel Protocol                           [ OK ]
Successfully established ngrok connection! (region: 'us', latency: 30.082413ms)

